# Getting available actions
Actions are configured in `app\configurations` directory. When the UI makes a request to get the actions available for an object, the `app/ajaxviews/actions.py` loads the yaml, and interprets what actions that the selected object is available to take. Those actions are sent to the user. 

### Two times when actions come up:
* web interaction that creates jobs
* azure functions that resolve actions

In [1]:
import sys
import numpy as np
import pandas as pd
import altair as alt

sys.path.append("..")

import yaml, ssl, asyncio

ssl._create_default_https_context = ssl._create_unverified_context
asyncio.set_event_loop_policy(asyncio.WindowsSelectorEventLoopPolicy())
import nest_asyncio

# this is required for running in a Jupyter Notebook.
nest_asyncio.apply()

import sys, os

from app.connectors import cmdb_graph
from app.objects import time as t

import app.ajaxviews.actions as actions
import app.models as models

c = models.CosmosdbClient()

from helpers import test_queries

executing local windows deployment


## web interaction that creates jobs

`app/ajaxviews/actions.py` takes input from the browser and creates jobs. Tools exist in that library to:
* validate which jobs an agent can do
* build the job and action graph elelements
* update the graph with the job

the resulting addition will be some kind of variant of:
`(action)<-job-(agent)`


In [15]:
# The pop here is our agent. We'll just grab one randomly. 
pop = test_queries.get_random_pop(c,'BillmanLocal2')
pop


{'isIdle': 'true',
 'name': 'Obattla Petown',
 'objid': '6577007174425',
 'conformity': 0.245,
 'literacy': 0.402,
 'aggression': 0.539,
 'constitution': 0.505,
 'health': 0.42,
 'isIn': '8822366435138',
 'industry': 0.317,
 'wealth': 0.2,
 'factionLoyalty': 0.483,
 'username': 'BillmanLocal2',
 'objtype': 'pop',
 'id': '6577007174425'}

You can load the full actions list. The actions is kept in a Yaml file. This comes from `app/configurations`. It's only used in the Ajax views as the action is stored in the graph, so no need to copy to the funciton. 

In [16]:
all_actions = actions.get_actions_config()
pd.DataFrame(all_actions)

,type,label,applies_to,effort,requires_attr,augments_self_properties,comment
0,healthcare_initiatives,action,pop,2,{'wealth': 0.1},"{'health': 0.05, 'wealth': '-.05'}",increase the health of your population slightl...
1,individual_education,action,pop,2,{'wealth': 0.1},"{'wealth': '-.1', 'literacy': 0.1, 'factionLoy...","expand privatized education, increase literacy..."
2,patriot_education,action,pop,1,{'factionLoyalty': 0.1},"{'factionLoyalty': 0.05, 'literacy': 0.1, 'agg...",expand public education programs with a collec...
3,patriot_propoganda,action,pop,1,NaN,"{'factionLoyalty': 0.1, 'conformity': 0.1, 'ag...","build national pride, increasin faction loyalt..."
4,build_infrastructure,action,pop,3,{'wealth': 0.5},"{'wealth': -0.1, 'industry': 0.1}",increase industry by investing in local infras...
5,build_building,action,pop,1,NaN,NaN,opens the building window for that population ...


`validator.validate()` returns the shotened list. of the actions that the population (or agent) can take. 

In [17]:
validator = actions.ActionValidator(pop, all_actions)
pd.DataFrame(validator.validate())

,type,label,applies_to,effort,requires_attr,augments_self_properties,comment
0,healthcare_initiatives,action,pop,2,{'wealth': 0.1},"{'health': 0.05, 'wealth': '-.05'}",increase the health of your population slightl...
1,individual_education,action,pop,2,{'wealth': 0.1},"{'wealth': '-.1', 'literacy': 0.1, 'factionLoy...","expand privatized education, increase literacy..."
2,patriot_education,action,pop,1,{'factionLoyalty': 0.1},"{'factionLoyalty': 0.05, 'literacy': 0.1, 'agg...",expand public education programs with a collec...
3,patriot_propoganda,action,pop,1,NaN,"{'factionLoyalty': 0.1, 'conformity': 0.1, 'ag...","build national pride, increasin faction loyalt..."
4,build_building,action,pop,1,NaN,NaN,opens the building window for that population ...


## Creating a job

to create a job you need an agent and and action. This is done in the ajax views for that class (e.g. `app/ajaxviews/pop`) because they are a request from the UI.



In [18]:
def create_job(pop, action, universalTime):
    if type(universalTime) == list:
        universalTime = universalTime[0]
    time_to_complete = int(universalTime["currentTime"]) + int(action["effort"])
    # not actually creating the node here, just demo.
    # uid = create_action_node(c,action,pop)
    popToAction = {
        "node1": pop["objid"],
        "node2": "1234567890",
        "label": "takingAction",
        "name": "takingAction",
        "weight": time_to_complete,
        "actionType": action["type"],
        "status": "pending",
    }
    edges = [popToAction]
    return edges


time = {"currentTime": 10000}
create_job(pop, all_actions[0], time)

[{'node1': '6577007174425',
  'node2': '1234567890',
  'label': 'takingAction',
  'name': 'takingAction',
  'weight': 10002,
  'actionType': 'healthcare_initiatives',
  'status': 'pending'}]

In [ ]:
time = 

In [14]:
actions_query = """
    g.V().haslabel('action').as('action')
                .inE('takingAction').has('status','pending').as('job')
                .outV().as('agent')
                .select('action','job','agent')
"""

c.run_query(actions_query)
action = c.res
action

[{'action': {'id': '515527ed-5a86-4a2f-be7f-19f9af45e23e',
   'label': 'action',
   'type': 'vertex',
   'properties': {'type': [{'id': 'de09040b-2c60-4a8a-b640-d78a248688f9',
      'value': 'healthcare_initiatives'}],
    'applies_to': [{'id': 'd753c296-7b62-4eb4-8e18-df39a67977ea',
      'value': 'pop'}],
    'effort': [{'id': '06664799-5d8b-452d-bfc3-ac8cf7cfaad6', 'value': 2}],
    'requires_attr': [{'id': 'd4265392-c925-461b-8367-79b7c14bbb89',
      'value': '{wealth: 0.1}'}],
    'augments_self_properties': [{'id': 'eaee7871-184e-4f20-b651-7ca1881c7e63',
      'value': '{health: 0.05, wealth: -.05}'}],
    'comment': [{'id': '43be8540-5b1e-4355-bab9-74f43f28be49',
      'value': 'increase the health of your population slightly, at the cost of a small amount of wealth'}],
    'name': [{'id': 'b08459e6-e4a3-4d53-87e2-b8890c3cc7b9',
      'value': 'healthcare initiatives'}],
    'objid': [{'id': '515527ed-5a86-4a2f-be7f-19f9af45e23e|objid',
      'value': '8522332346729'}],
    'us

All actions require both an agent, job and action. The properties of the action determine how it affects other things. 


### Augments self properties
Takes a dictionary of properties and augments them. Adds the property if it does not exist. 


In [16]:
augments_self_properties = yaml.safe_load(action["augments_self_properties"])
augments_self_properties

NameError: name 'action' is not defined

In [94]:
def resolve_augments_self_properties(agent, action):
    agent = agent.copy()
    self_properties = yaml.safe_load(action["augments_self_properties"])
    for p in self_properties.keys():
        agent[p] = agent[p] + float(self_properties[p])
    return agent


agent2 = resolve_augments_self_properties(agent, action)
results = pd.DataFrame([agent, agent2],).T
print("results of: ", action["type"])
results

results of:  individual_education


,0,1
isIdle,True,True
name,Letquamar Mennu,Letquamar Mennu
objid,2570162977799,2570162977799
conformity,0.527,0.527
literacy,0.473,0.483
aggression,0.442,0.442
constitution,0.576,0.576
health,0.5,0.5
isInFaction,6048684961165,6048684961165
industry,0.509,0.509


In [100]:
def query_patch_properties(agent, action):
    query = f"g.V().has('objid','{agent['objid']}')"
    for n in yaml.safe_load(action["augments_self_properties"]):
        query += f".property('{n}',{agent[n]})"
    return query


query_patch_properties(agent, action)

"g.V().has('objid','2570162977799').property('wealth',0.491).property('literacy',0.473).property('factionLoyalty',0.609)"

Veryify that actions are resolved or not

In [105]:
actions_query = """
    g.V().haslabel('action').as('action')
                .inE('takingAction').has('status','resolved').as('job')
                .outV().as('agent')
                .select('action','job','agent')
"""

c.run_query(actions_query)
pd.DataFrame(c.res)

,action,job,agent
0,"{'id': '539f4d55-7dea-487a-8b62-756b0fe0d313',...","{'id': '6e029d81-83c9-4069-afa6-639aef48bd8f',...","{'id': 'eaa24c0e-8dcf-43d5-af89-ca6bb91d3111',..."
1,"{'id': '42011202-a68d-43d8-841f-87c50f42ac58',...","{'id': '5e9dce26-a02b-4972-a1c3-97d1f7d85f16',...","{'id': '083f7cac-dade-4961-b554-ad3009e704fa',..."
2,"{'id': 'c08bb02d-5739-49f4-a8c1-62c905f0bee5',...","{'id': 'e528b285-0e02-41a4-9226-0b94ffb8433c',...","{'id': '0f6f0e7b-4ce4-4390-a455-5f24a859a62d',..."
3,"{'id': 'aa2e3217-0a2b-47e9-abf7-c963ac28a543',...","{'id': 'e3fa5d2e-ce10-40dc-9367-9e95c3bba555',...","{'id': '90bcb328-de86-42d4-a59d-c182cc3eb995',..."
